In [32]:
import duckdb 
import pandas as pd
import plotly.express as px
import plotly
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

In [33]:
con = duckdb.connect('load_results.duckdb')
scale_db_names = [
    'benchmark_log_python_load_test_030_027.db',
    'benchmark_log_python_load_test_032_031_partial_030.db',
    'benchmark_log_python_load_test_071_034.db',
    'benchmark_log_python_load_test_090_081.db',
    'benchmark_log_python_load_test_092_091.db',
    'benchmark_log_python_1e9_success_0_10_on.db',
    # Not sure about this one:
    'benchmark_log_python_1e9_join_success.db',
    'benchmark_log_python_load_test_0103_100_latest.db',
    ]
for i, scale_db_name in enumerate(scale_db_names):
    try:
        detach_results = con.sql(f"""DETACH "{scale_db_name}" """)    
    except:
        print(f"Unable to detach {scale_db_name} - it may not be attached yet")
    attach_results = con.sql(f"""ATTACH '{scale_db_name}' as "{scale_db_name}" (TYPE SQLITE)""")
    if i == 0:
        ctas_results = con.sql(f"""
                            create or replace table duckdb_results as 
                            select '' as filename, * from "{scale_db_name}".results where 1=0""")
    insert_results = con.sql(f"""
        insert into duckdb_results 
            select '{scale_db_name}' as filename, * from "{scale_db_name}".results
        """)



In [34]:
con.sql("""from duckdb_results""").df()

,filename,run_id,repeat_id,benchmark,scenario,time
0,benchmark_log_python_load_test_030_027.db,1,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""0.3.0"", ""row_count"": ""1e8""}",59.533625
1,benchmark_log_python_load_test_030_027.db,1,0,103 Group By Scale test: Group by queries,"{""duckdb_version"": ""0.3.0"", ""row_count"": ""1e8""}",268.623644
2,benchmark_log_python_load_test_030_027.db,1,0,201 Join Scale test: Create tables from csvs j...,"{""duckdb_version"": ""0.3.0"", ""row_count"": ""1e8""}",107.957932
3,benchmark_log_python_load_test_030_027.db,1,0,203 Join Scale test: Join queries,"{""duckdb_version"": ""0.3.0"", ""row_count"": ""1e8""}",4285.359142
4,benchmark_log_python_load_test_030_027.db,2,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""0.2.9"", ""row_count"": ""1e8""}",56.876784
5,benchmark_log_python_load_test_030_027.db,2,0,103 Group By Scale test: Group by queries,"{""duckdb_version"": ""0.2.9"", ""row_count"": ""1e8""}",285.871582
6,benchmark_log_python_load_test_030_027.db,2,0,201 Join Scale test: Create tables from csvs j...,"{""duckdb_version"": ""0.2.9"", ""row_count"": ""1e8""}",106.528665
7,benchmark_log_python_load_test_030_027.db,2,0,203 Join Scale test: Join queries,"{""duckdb_version"": ""0.2.9"", ""row_count"": ""1e8""}",3685.714231
8,benchmark_log_python_load_test_030_027.db,3,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""0.2.8"", ""row_count"": ""1e8""}",52.141962
9,benchmark_log_python_load_test_030_027.db,3,0,103 Group By Scale test: Group by queries,"{""duckdb_version"": ""0.2.8"", ""row_count"": ""1e8""}",332.031949


In [35]:
con.sql("""
                 create or replace table refined_results as
        with version_ranks as (
        from duckdb_results
        select distinct
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            version.string_split('.') as split_version,
            try_cast(split_version[1] as int) * 100 * 100 as first_digit,
            try_cast(split_version[2] as int) * 100 as second_digit,
            try_cast((split_version[3]).string_split('-')[1] as int) as third_digit,
            first_digit + second_digit + third_digit as version_rank,
    ), max_version as (
        from version_ranks
        select 
            max(version_rank),
            arg_max(version, version_rank) as max_version
    )         
        from duckdb_results
        join version_ranks on (duckdb_results.scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') = version_ranks.version
                 select 
                    (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as duckdb_version,
                    scenario::json ->> 'row_count' as row_count,
                    duckdb_results.*,
                    version_ranks.version_rank
                 where not (filename = 'benchmark_log_python_load_test_032_031_partial_030.db' and duckdb_version = '0.3.0')
                """)
raw_df = con.sql("""from refined_results order by version_rank desc""").df()
raw_df.head(10000)

,duckdb_version,row_count,filename,run_id,repeat_id,benchmark,scenario,time,version_rank
0,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,203 Join Scale test: Join queries,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",1932.606873,10001
1,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,202 Join Scale test: Convert to Enums for joins,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",516.170051,10001
2,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,201 Join Scale test: Create tables from csvs j...,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",856.983232,10001
3,1.0.1-dev1125,1e8,benchmark_log_python_load_test_0103_100_latest.db,3,0,203 Join Scale test: Join queries,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",293.182044,10001
4,1.0.1-dev1125,1e8,benchmark_log_python_load_test_0103_100_latest.db,3,0,202 Join Scale test: Convert to Enums for joins,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",22.444009,10001
5,1.0.1-dev1125,1e8,benchmark_log_python_load_test_0103_100_latest.db,3,0,201 Join Scale test: Create tables from csvs j...,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",39.375726,10001
6,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,103 Group By Scale test: Group by queries,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",478.377793,10001
7,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,102 Group By Scale test: Convert to Enums,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",228.198938,10001
8,1.0.1-dev1125,1e9,benchmark_log_python_load_test_0103_100_latest.db,3,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",242.097343,10001
9,1.0.1-dev1125,1e8,benchmark_log_python_load_test_0103_100_latest.db,3,0,103 Group By Scale test: Group by queries,"{""duckdb_version"": ""v1.0.1-dev1125"", ""row_coun...",13.330142,10001


In [52]:
summary = con.sql("""
                with version_combos as (
                  select * 
                  from (select distinct version_rank, duckdb_version from refined_results)
                  cross join (select distinct row_count from refined_results)
                ), summary as (
                    from version_combos
                    left join refined_results 
                        on version_combos.duckdb_version = refined_results.duckdb_version
                        and version_combos.row_count = refined_results.row_count
                    
                    select 
                        version_combos.version_rank,
                        version_combos.duckdb_version,
                        version_combos.row_count as row_count_text,
                        case when version_combos.row_count = '1e8' then 100000000 else 1000000000 end as row_count,
                        case when sum(case when benchmark = '203 Join Scale test: Join queries' then 1 else 0 end) >= 1 then 1 else 0 end as "Join",
                        case when sum(case when benchmark = '103 Group By Scale test: Group by queries' then 1 else 0 end) >= 1 then 1 else 0 end as "Group By",
                    
                    group by all
                    order by
                        version_combos.version_rank desc,
                        version_combos.row_count asc
                ), unpivoted as (
                  unpivot summary 
                    on "Join", "Group By"
                    into 
                      name query_type
                      value success
                )
                from unpivoted 
                select 
                    version_rank,
                  duckdb_version as "DuckDB Version",
                  query_type,
                  coalesce(max(row_count) filter(success = 1),10000000) as max_row_count
                group by all
                order by 
                  query_type,  
                  version_rank asc
                    
                  
                
""").df()
summary

,version_rank,DuckDB Version,query_type,max_row_count
0,207,0.2.7,Group By,100000000
1,208,0.2.8,Group By,100000000
2,209,0.2.9,Group By,100000000
3,300,0.3.0,Group By,100000000
4,301,0.3.1,Group By,100000000
5,302,0.3.2,Group By,100000000
6,304,0.3.4,Group By,100000000
7,400,0.4.0,Group By,100000000
8,501,0.5.1,Group By,10000000
9,601,0.6.1,Group By,100000000


In [53]:
# How long did the longest scale take?
large_results = con.sql("""
                        from raw_df 
                        join summary 
                            on raw_df.duckdb_version = summary."DuckDB Version" 
                            and (
                                ( 
                                    (raw_df.benchmark ilike '%join scale test%') 
                                    and (raw_df.benchmark ilike '%join scale test%') = (summary.query_type ilike '%join%')
                                )
                                or
                                (
                                    (raw_df.benchmark ilike '%group by scale test%')
                                    and (raw_df.benchmark ilike '%group by scale test%') = (summary.query_type ilike '%group%')
                                )
                            )
                        select 
                            raw_df.*
                        where 
                            row_count = '1e9'
                            and summary.max_row_count = 1000000000
                        order by
                            benchmark,
                            raw_df.version_rank
                        """).df()
large_results

,duckdb_version,row_count,filename,run_id,repeat_id,benchmark,scenario,time,version_rank
0,0.9.0,1e9,benchmark_log_python_load_test_092_091.db,3,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""0.9.0"", ""row_count"": ""1e9""}",183.060313,900
1,0.9.0,1e9,benchmark_log_python_load_test_090_081.db,1,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""0.9.0"", ""row_count"": ""1e9""}",164.543518,900
2,0.9.1,1e9,benchmark_log_python_load_test_092_091.db,2,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.9.1"", ""row_count"": ""1e9""}",188.836829,901
3,0.9.1,1e9,benchmark_log_python_1e9_success_0_10_on.db,5,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.9.1"", ""row_count"": ""1e9""}",191.805675,901
4,0.9.2,1e9,benchmark_log_python_1e9_success_0_10_on.db,4,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.9.2"", ""row_count"": ""1e9""}",152.937549,902
5,0.9.2,1e9,benchmark_log_python_load_test_092_091.db,1,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.9.2"", ""row_count"": ""1e9""}",155.136888,902
6,0.10.0,1e9,benchmark_log_python_1e9_success_0_10_on.db,3,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.10.0"", ""row_count"": ""1e9""}",217.309982,1000
7,0.10.1,1e9,benchmark_log_python_1e9_success_0_10_on.db,2,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.10.1"", ""row_count"": ""1e9""}",241.120261,1001
8,0.10.1,1e9,benchmark_log_python_1e9_join_success.db,2,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.10.1"", ""row_count"": ""1e9""}",232.555970,1001
9,0.10.2,1e9,benchmark_log_python_1e9_success_0_10_on.db,1,0,101 Group By Scale test: Create table from csv,"{""duckdb_version"": ""v0.10.2"", ""row_count"": ""1e9""}",242.981022,1002


In [54]:
fig = px.bar(summary, 
             x='DuckDB Version', 
             y='max_row_count',
             color='query_type',
            #  barmode='group',
             template='plotly_white',
             color_discrete_sequence=px.colors.qualitative.T10,
             range_y=[0,1000000000],
             facet_row='query_type',
            #  facet_row_spacing=0.3,
             )

fig.show()


In [55]:
# Also plot by date
from datetime import datetime
versions = {
    # 0.2.7 is the first with MacOS ARM
    '0.2.7': {'date':datetime.strptime('2021-06-14','%Y-%m-%d')},
    '0.2.8': {'date':datetime.strptime('2021-08-02','%Y-%m-%d')},
    '0.2.9': {'date':datetime.strptime('2021-09-06','%Y-%m-%d')},
    '0.3.0': {'date':datetime.strptime('2021-10-06','%Y-%m-%d')},
    '0.3.1': {'date':datetime.strptime('2021-11-16','%Y-%m-%d')},
    '0.3.2': {'date':datetime.strptime('2022-02-07','%Y-%m-%d')},
    # 0.3.3 did not upload to pip correctly so it should be skipped
    # '0.3.3': {'date':datetime.strptime('2022-04-11','%Y-%m-%d')},
    '0.3.4': {'date':datetime.strptime('2022-04-25','%Y-%m-%d')},
    '0.4.0': {'date':datetime.strptime('2022-06-20','%Y-%m-%d')},
    '0.5.1': {'date':datetime.strptime('2022-09-19','%Y-%m-%d')},
    '0.6.1': {'date':datetime.strptime('2022-12-06','%Y-%m-%d')},
    '0.7.1': {'date':datetime.strptime('2023-02-27','%Y-%m-%d')},
    '0.8.1': {'date':datetime.strptime('2023-06-13','%Y-%m-%d')},
    '0.9.0': {'date':datetime.strptime('2023-09-26','%Y-%m-%d')},
    '0.9.1': {'date':datetime.strptime('2023-10-11','%Y-%m-%d')},
    '0.9.2': {'date':datetime.strptime('2023-11-14','%Y-%m-%d')},
    '0.10.0': {'date':datetime.strptime('2024-02-13','%Y-%m-%d')},
    '0.10.1': {'date':datetime.strptime('2024-03-18','%Y-%m-%d')},
    '0.10.2': {'date':datetime.strptime('2024-04-17','%Y-%m-%d')},
    '0.10.3': {'date':datetime.strptime('2024-05-22','%Y-%m-%d')},
    '1.0.0': {'date':datetime.strptime('2024-06-03','%Y-%m-%d')},
    '1.0.1-dev1125': {'date':datetime.now()},
}
versions_df = pd.DataFrame([versions])
# versions_df
plus_dates = con.sql("""
    with unpivoted as (
        unpivot versions_df
        on columns(*)
        into name version
             value date_struct
    ), version_to_date_map as (
        from unpivoted 
        select 
            version,
            date_struct.date as "Release Date"
    )
    from summary data
    left join version_to_date_map on data."DuckDB Version" = version_to_date_map.version
    select 
        data.*,
        data.max_row_count as "Max Row Count",
        data.query_type as "Benchmark",
        version_to_date_map.* exclude version
        
""").df()
# plus_dates

In [79]:
fig = px.bar(plus_dates, 
             x='Release Date', 
             y='Max Row Count',
             color='Benchmark',
            #  barmode='group',
             template='plotly_white',
             color_discrete_sequence=px.colors.qualitative.T10,
             range_y=[0,1000000000],
             facet_row='Benchmark',
             hover_data=['DuckDB Version'],

             )
# fig = fig.update_traces(width=[0.5]*len(versions.keys()), overwrite=True)

print(plotly.io.to_json(fig, pretty=True))

# fig.update_layout()
fig.show()


{
  "data": [
    {
      "alignmentgroup": "True",
      "customdata": [
        [
          "0.2.7"
        ],
        [
          "0.2.8"
        ],
        [
          "0.2.9"
        ],
        [
          "0.3.0"
        ],
        [
          "0.3.1"
        ],
        [
          "0.3.2"
        ],
        [
          "0.3.4"
        ],
        [
          "0.4.0"
        ],
        [
          "0.5.1"
        ],
        [
          "0.6.1"
        ],
        [
          "0.7.1"
        ],
        [
          "0.8.1"
        ],
        [
          "0.9.0"
        ],
        [
          "0.9.1"
        ],
        [
          "0.9.2"
        ],
        [
          "0.10.0"
        ],
        [
          "0.10.1"
        ],
        [
          "0.10.2"
        ],
        [
          "0.10.3"
        ],
        [
          "1.0.0"
        ],
        [
          "1.0.1-dev1125"
        ]
      ],
      "hovertemplate": "Benchmark=Group By\u003cbr\u003eRelease Date=%{x}\u003cbr\u003e

In [57]:
fig = px.line(plus_dates, 
             x='Release Date', 
             y='max_row_count',
             color='query_type',
             hover_data='DuckDB Version',
            #  barmode='group',
             template='plotly_white',
             color_discrete_sequence=px.colors.qualitative.T10,
             range_y=[0,1000000000],
             facet_row='query_type',
             line_shape='hv'
             )

fig.show()
